In [ ]:
import os
print(os.getcwd())

In [1]:
import cv2
from ultralytics import YOLO

In [ ]:
#split frames 

import cv2
import numpy as np

def split_video_skip_frames(input_video_path, output_folder, skip_frames):
    """
    Splits a video into individual frames, skipping a specified number of frames.

    :param input_video_path: Path to the input video.
    :param output_folder: Folder to save the extracted frames.
    :param skip_frames: Number of frames to skip.
    """
    # Open the video file
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_idx = 0
    saved_frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames
        if frame_idx % (skip_frames + 1) == 0:
            output_filename = f"{output_folder}/frame_{saved_frame_idx}.jpg"
            cv2.imwrite(output_filename, frame)
            saved_frame_idx += 1

        frame_idx += 1

    cap.release()

# Example usage
split_video_skip_frames("sample.mp4", "Frames2", 10)  # Adjust the path and skip_frames


In [ ]:
# Load a pretrained model
model = YOLO("yolov8n.pt")

# Train the model using your .yaml file
model.train(data="Frames/data.yaml", epochs=50)


In [ ]:
# Load a pretrained model
model = YOLO("yolov8n.pt")

# Train the model using your .yaml file
model.train(data="Frames2/data.yaml", epochs=50)


In [3]:
def boxes_overlap(result1, result2, i):
    
    if len(result1.boxes.xyxy) == 0 or len(result2.boxes.xyxy) == 0:
        return False
        
    # Access the bounding boxes from the Results objects
    box1 = result1.boxes.xyxy[0].cpu().numpy()  # Box 1
    print("box1: ", box1)
    box2 = result2.boxes.xyxy[i].cpu().numpy()  # Box 2

    # Unpack the bounding boxes
    x1_min, y1_min, x1_max, y1_max = box1
    
    # Shrink Box 2 by a certain factor (to reduce chances of tag getting included in other human bounding box)
    shrink_factor = 0.2
    x2_min, y2_min, x2_max, y2_max = box2
    width = x2_max - x2_min
    height = y2_max - y2_min
    shrink_x = width * shrink_factor
    shrink_y = height * shrink_factor
    x2_min += shrink_x
    y2_min += shrink_y
    x2_max -= shrink_x
    y2_max -= shrink_y

    # Check if Box 1 is within the shrunk Box 2
    if x1_min >= x2_min and x1_max <= x2_max and y1_min >= y2_min and y1_max <= y2_max:
        return True

# Set to keep track of staff member IDs
staff_ids = set()

# Initialize a dictionary to store timestamps and coordinates for each staff member
staff_data = {}

# Load the YOLOv8 models
human_model = YOLO('people_model.pt')
tag_model = YOLO('tag_model.pt')

# Open the video file
video_path = "sample.mp4"
cap = cv2.VideoCapture(video_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # or 'XVID', 'MJPG', 'AVI', etc.
out = cv2.VideoWriter('output2.mp4', fourcc, 20.0, (960, 720))  # Adjust the frame rate and frame size as needed

while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Get the current timestamp in the video
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC)

        # Run YOLOv8 tracking for tag and human detection
        tag_results = tag_model.track(frame, persist=True)
        human_results = human_model.track(frame, persist=True)
 
        # Check each tag if it's inside any human bounding box
        for tag in tag_results:
            for human in human_results:
                for i in range(len(human.boxes)):
                    if boxes_overlap(tag, human,i):
                        human_id = human.boxes[i].id.item()
                        #add staff_id into set
                        staff_ids.add(human_id)
                        break  # Assuming one tag per person, break the loop

        
        # Record timestamp for each identified staff member
        for human in human_results:
            # Check if human.boxes.id is not None
            if human.boxes.id is not None:
                for i, _id in enumerate(human.boxes.id):
                    human_id = _id.item()  # Convert tensor to scalar
                    if human_id in staff_ids:
                        # Extract the coordinates (center of the box)
                        x_center = int((human.boxes.xyxy[i][0] + human.boxes.xyxy[i][2]) / 2)
                        y_center = int((human.boxes.xyxy[i][1] + human.boxes.xyxy[i][3]) / 2)
    
                        # Display the coordinates
                        cv2.putText(frame, f"Staff id: {human_id}, Coordinates X,Y: {x_center}, {y_center}", 
                                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                                    0.7, (255, 0, 0), 2)
                        
                        # Initialize the list for this staff_id if it doesn't exist
                        if human_id not in staff_data:
                            staff_data[human_id] = []
                            
                        # Append the timestamp and coordinates
                        staff_data[human_id].append((current_time, x_center, y_center))

        # Visualize the results on the frame
        annotated_frame = tag_results[0].plot()
        annotated_frame2 = human_results[0].plot()
         # Combine annotated frames
        combined_frame = cv2.addWeighted(annotated_frame, 0.5, annotated_frame2, 0.5, 0)

        # Write the processed frame to the video file
        out.write(combined_frame)
        
        # Display the combined annotated frame
        cv2.imshow("YOLOv8 Combined Tracking", combined_frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 11 peoples, 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 11 peoples, 7.9ms
Speed: 1.0ms preprocess, 7.9ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 11 peoples, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 12 peoples, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.5ms postprocess per imag

In [7]:
print(staff_data)

{22.0: [(15040.000000000002, 512, 76), (15080.0, 519, 78), (15120.000000000002, 525, 82), (15160.0, 527, 84), (15200.000000000002, 524, 85), (15240.0, 528, 85), (15280.000000000002, 532, 88), (15320.0, 533, 102), (15360.0, 533, 115), (15400.0, 534, 128), (15440.0, 536, 138), (15480.0, 540, 169), (15520.0, 549, 195), (15560.0, 556, 207), (15600.0, 563, 237), (15640.0, 566, 270), (15680.0, 567, 299), (15720.0, 562, 334), (15760.0, 555, 350), (15800.0, 565, 384), (15840.0, 568, 417), (15880.0, 570, 469), (15920.0, 560, 498), (15960.0, 551, 510), (16000.0, 547, 523), (16040.0, 536, 554), (16080.000000000002, 529, 573), (16120.000000000002, 523, 586), (16160.0, 520, 600), (16200.0, 521, 617), (16240.000000000002, 520, 627), (16280.000000000002, 520, 631), (16320.0, 519, 633), (16360.0, 517, 643), (16400.0, 515, 652), (16440.0, 514, 655), (16480.0, 517, 656), (16520.0, 519, 666), (16560.0, 519, 672), (16600.0, 517, 673), (16640.0, 514, 673), (16880.0, 512, 663), (16920.0, 512, 662), (16960.0

In [9]:
#write into CSV

import pandas as pd

# Flatten the staff_data into a list of rows
rows = []
for staff_id, data in staff_data.items():
    for record in data:
        timestamp_ms, x, y = record
        timestamp_sec = timestamp_ms / 1000  # Convert milliseconds to seconds
        rows.append([staff_id, timestamp_sec, x, y])

# Create a DataFrame and write to CSV/Excel
df = pd.DataFrame(rows, columns=['Staff ID', 'Timestamp (s)', 'X Coordinate', 'Y Coordinate'])

# Write to CSV
df.to_csv('staff_data.csv', index=False)

